In [1]:
import torch
import matplotlib.pyplot as plt

In [4]:
model = torch.nn.Sequential(
    torch.nn.Linear(1, 20),
    torch.nn.Tanh(),
    torch.nn.Linear(20, 1),
)
model.load_state_dict(torch.load('tanh-20.pth'))

<All keys matched successfully>

In [5]:
model

Sequential(
  (0): Linear(in_features=1, out_features=20, bias=True)
  (1): Tanh()
  (2): Linear(in_features=20, out_features=1, bias=True)
)